In [1]:
import numpy as np
import pandas as pd

In [2]:
roi_damage = pd.read_csv('/home/despoB/dlurie/Projects/despolab_lesion/derivatives/sub-101/anat/sub-101_t1W_atlas-BNA_mask-lesion_ROIdamage.csv', index_col='roi_label')

In [3]:
roi_list = np.loadtxt('/home/despoB/dlurie/Projects/despolab_lesion/analysis/first_pass_ROIs.txt', dtype='int')
roi_list = roi_list[1:]

In [4]:
keep_df = roi_damage.loc[roi_list]

In [5]:
damaged_rois = keep_df[keep_df.pct_lesion > 1].index.values

In [6]:
damaged_rois

array([ 36,  38,  44,  46,  52, 166, 168, 220, 224, 228])

In [7]:
damaged_roi_idxs = damaged_rois - 1

In [8]:
dmg_pcts = roi_damage.loc[damaged_rois].pct_lesion.values

In [9]:
dmg_pcts

array([ 21.46523565,  24.29993777,  67.18916993,  25.42047802,
        68.36544438,  22.57696693,   1.4084507 ,   5.89695841,
         4.6852123 ,   2.689037  ])

In [26]:
pc = np.loadtxt('/home/despoB/dlurie/Projects/despolab_lesion/sandbox/hcp_mean_pc-louvain_gamma-1.0.txt')

In [27]:
wd = np.loadtxt('/home/despoB/dlurie/Projects/despolab_lesion/sandbox/hcp_mean_wmdz-louvain_gamma-1.0.txt')

In [11]:
pc_vals = pc[damaged_roi_idxs]

In [12]:
pc_vals

array([ 0.62218371,  0.61945266,  0.20197738,  0.59666479,  0.44499332,
        0.63075904,  0.63902132,  0.33764873,  0.48524963,  0.36325416])

In [13]:
dmg_all = dmg_pcts * pc_vals

In [14]:
pc_damage_score = np.mean(dmg_all)

In [15]:
pc_damage_score

10.795040130696616

In [16]:
def filter_rois(patient_damage_df, common_rois, damage_thresh):
    common_df = patient_damage_df.loc[common_rois]
    damaged_rois = common_df[common_df.pct_lesion > damage_thresh].index.values
    return damaged_rois

In [17]:
knockout = filter_rois(roi_damage, roi_list, 1)

In [30]:
def pc_damage(patient_damage_df, healthy_pc, damaged_rois):
    damage_pcts = patient_damage_df.loc[damaged_rois].pct_lesion.values
    pc_vals = healthy_pc[damaged_rois - 1]
    damage_score = np.mean(damage_pcts * pc_vals)
    return damage_score

In [32]:
def wd_damage(patient_damage_df, healthy_wd, damaged_rois):
    damage_pcts = patient_damage_df.loc[damaged_rois].pct_lesion.values
    wd_vals = healthy_wd[damaged_rois - 1]
    damage_score = np.mean(damage_pcts * wd_vals)
    return damage_score

In [31]:
pc_damage(roi_damage, pc, knockout)

10.795040130696616

In [33]:
wd_damage(roi_damage, wd, knockout)

4.5382951705617627

In [34]:
# Load list of subjects 
sublist_file = '/home/despoB/dlurie/Projects/despolab_lesion/derivatives/has_EPI.txt'
subject_list = np.loadtxt(sublist_file, dtype='int')
subject_list = [str(i) for i in subject_list]

In [35]:
skip = np.loadtxt('/home/despoB/dlurie/Projects/despolab_lesion/analysis/first_pass_skips.txt', dtype='int')
skip = [str(i) for i in skip]

In [36]:
damage_fpt = '/home/despoB/dlurie/Projects/despolab_lesion/derivatives/sub-{0}/anat/sub-{0}_t1W_atlas-BNA_mask-lesion_ROIdamage.csv'

In [38]:
damage_scores = []
for pid in subject_list:
    if pid not in skip:
        patient_damage_df = pd.read_csv(damage_fpt.format(pid), index_col='roi_label')
        knockout = filter_rois(patient_damage_df, roi_list, 0)
        pc_damage_score = pc_damage(patient_damage_df, pc, knockout)
        wd_damage_score = wd_damage(patient_damage_df, wd, knockout)
        damage_scores.append((pid, pc_damage_score, wd_damage_score))

In [39]:
damage_scores_df = pd.DataFrame.from_records(damage_scores, columns=['pid', 'pc_damage', 'wd_damage'])

In [40]:
damage_scores_df.to_csv('/home/despoB/dlurie/Projects/despolab_lesion/analysis/damage_all.csv', index=False)